In [ ]:
!pip install -qU transformers peft trl accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import json
import re
import os

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from huggingface_hub import notebook_login, HfFolder, HfApi
from datasets import load_dataset, DatasetDict, Dataset
# from dotenv import load_dotenv

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from accelerate import Accelerator
import bitsandbytes as bnb
from trl import SFTTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM

In [ ]:

DATASET_ID = "Sorour/sentiment-train" # "Sorour/sentiment-train", "Sorour/headline","Sorour/fomc"
df = load_dataset(DATASET_ID)

Generating train split:   0%|          | 0/3850 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/964 [00:00<?, ? examples/s]

In [ ]:
icl_data = df['train']
icl_df = pd.DataFrame(icl_data)

In [ ]:
# Function to randomly select a specified number of examples from each label
def select_random_examples(df, label_column, num_samples):
    selected_examples = []
    labels = df[label_column].unique()
    for label in labels:
        examples = df[df[label_column] == label]
        # Ensure we don't sample more than available examples
        samples_to_draw = min(num_samples, len(examples))
        selected_example = examples.sample(n=samples_to_draw)
        selected_examples.append(selected_example)
    return pd.concat(selected_examples).reset_index(drop=True)

# Specify the number of samples to retrieve for each label
num_samples = 1  # Change this value as needed

# Select random examples based on the 'label' column and the specified number of samples
random_examples = select_random_examples(icl_df, 'label', num_samples)

# Display the selected random examples
random_examples

,label,text,instruction,type
0,neutral,The availability of the Internet services is h...,You are a skilled financial analyst specialize...,sentiment
1,positive,The company also expects to sell its products ...,You are a skilled financial analyst specialize...,sentiment
2,negative,Aviva Fined $27 Million by U.K. Regulator Over...,You are a skilled financial analyst specialize...,sentiment


In [ ]:
def generate_dynamic_prompt_template(insruction,num_examples):
    prompt_template = """\
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    {instruction}. See below a couple of examples\n"""
    for i in range(1, num_examples + 1):
        prompt_template += f"<|eot_id|><|start_header_id|>user<|end_header_id|>[SENT]{{text{i}}}[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n{{label{i}}}\n"
    prompt_template += "[SENT]{Context}[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    return prompt_template

# Function to generate prompts based on the selected examples
def generate_prompt_with_samples(df, text_column, label_column, instruction_column,num_samples):
    # Generate dynamic prompt template based on the number of samples
    INSTRUCTION_PROMPT_TEMPLATE = generate_dynamic_prompt_template(df.loc[0,'instruction'],num_samples)

    RESPONSE_TEMPLATE = """\
    {answer}"""
    # Ensure there are enough examples
    if len(df) < num_samples:
        raise ValueError(f"The DataFrame must contain at least {num_samples} examples for each label.")
    #samples = df.sample(n=num_samples).reset_index(drop=True)
    # Create a dictionary to store the values to be formatted into the template
    format_dict = {}
    format_dict['instruction'] = df.loc[0, instruction_column]  # Use instruction from the first row
    for i in range(num_samples):
        format_dict[f'text{i + 1}'] = df.loc[i, text_column]
        format_dict[f'label{i + 1}'] = df.loc[i, label_column]
    format_dict['Context'] = "{Context}"  #

    # Generate the prompt with the samples
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(**format_dict)

    return prompt

# Specify the number of samples to retrieve for each label
num_samples = 1  # Change this value as needed

# Select random examples based on the 'label' column and the specified number of samples
random_examples = select_random_examples(icl_df, 'label', num_samples)

# Generate prompt with the selected examples
prompt_with_samples = generate_prompt_with_samples(random_examples, 'text', 'label', 'instruction',len(random_examples))

# Display the generated prompt with placeholders for text samples
print("Generated Prompt with Samples:")
print(prompt_with_samples)

Generated Prompt with Samples:
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    you are a skilled financial analyst specialized in detecting market sentiment from news sources.Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.
 Return only a single word, either Positive or Negative or Neutral. See below a couple of examples
<|eot_id|><|start_header_id|>user<|end_header_id|>[SENT]The facility will be used to refinance Citycon 's existing credit facility , the company said .[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>
neutral
<|eot_id|><|start_header_id|>user<|end_header_id|>[SENT]Sponda Plc Stock Exchange Release 5 December 2006 at 15.45 EUR 1.5 BILLION SYNDICATED CREDIT FACILITY FOR SPONDA Sponda has signed an agreement for a one-year syndicated credit facility for a total of EUR 1.5 billion .[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>
positive
<|

In [ ]:
prompt_with_samples.format(Context="test mikonim")

"    <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n    you are a skilled financial analyst specialized in detecting market sentiment from news sources.Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.\n Return only a single word, either Positive or Negative or Neutral. See below a couple of examples\n<|eot_id|><|start_header_id|>user<|end_header_id|>[SENT]The facility will be used to refinance Citycon 's existing credit facility , the company said .[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nneutral\n<|eot_id|><|start_header_id|>user<|end_header_id|>[SENT]Sponda Plc Stock Exchange Release 5 December 2006 at 15.45 EUR 1.5 BILLION SYNDICATED CREDIT FACILITY FOR SPONDA Sponda has signed an agreement for a one-year syndicated credit facility for a total of EUR 1.5 billion .[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\npositive\n<|eot_id|><|start_header_

In [ ]:
def generate_dynamic_prompt_template(insruction,num_examples):
    prompt_template = """\
    <s>[INST] <<SYS>>
    {instruction}<</SYS>>. See below a couple of examples:\n"""
    for i in range(1, num_examples + 1):
      # if i==1:
      #   prompt_template += f"Sentence:{{text{i}}} \n Label:[/INST]{{label{i}}}\n"
        prompt_template += f"Sentence:{{text{i}}} \n Label:{{label{i}}}\n"
    prompt_template += "Sentence:{Context} \n Label:[/INS]"
    return prompt_template

# Function to generate prompts based on the selected examples
def generate_prompt_with_samples(df, text_column, label_column, instruction_column,num_samples):
    # Generate dynamic prompt template based on the number of samples
    INSTRUCTION_PROMPT_TEMPLATE = generate_dynamic_prompt_template(df.loc[0,'instruction'],num_samples)

    RESPONSE_TEMPLATE = """\
    {answer}"""
    # Ensure there are enough examples
    if len(df) < num_samples:
        raise ValueError(f"The DataFrame must contain at least {num_samples} examples for each label.")

    # Create a dictionary to store the values to be formatted into the template
    format_dict = {}
    format_dict['instruction'] = df.loc[0, instruction_column]  # Use instruction from the first row
    for i in range(num_samples):
        format_dict[f'text{i + 1}'] = df.loc[i, text_column]
        format_dict[f'label{i + 1}'] = df.loc[i, label_column]
    format_dict['Context'] = "{Context}"  #

    # Generate the prompt with the samples
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(**format_dict)

    return prompt

# Specify the number of samples to retrieve for each label
num_samples = 1  # Change this value as needed

# Select random examples based on the 'label' column and the specified number of samples
random_examples = select_random_examples(icl_df, 'label', num_samples)

# Generate prompt with the selected examples
prompt_with_samples = generate_prompt_with_samples(random_examples, 'text', 'label', 'instruction',len(random_examples))

# Display the generated prompt with placeholders for text samples
print("Generated Prompt with Samples:")
print(prompt_with_samples)

Generated Prompt with Samples:
    <s>[INST] <<SYS>>
    You are a skilled financial analyst specialized in detecting market sentiment from news sources.Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.
 Return only a single word, either Positive or Negative or Neutral<</SYS>>. See below a couple of examples:
Sentence:Its annual capacity is some 10,000 MW . 
 Label:neutral
Sentence:Finnair expands : Finnair plans to add more flights on routes to Hong Kong , India and Russia . 
 Label:positive
Sentence:And U.S. energy executives say high steel prices are threatening energy exploration . 
 Label:negative
Sentence:{Context} 
 Label:[/INS]


In [ ]:
def generate_dynamic_prompt_template(insruction,num_examples):
    prompt_template = """\
    <|user|>
    {instruction}. See below a couple of examples\n"""
    for i in range(1, num_examples + 1):
        prompt_template += f"Sentence:{{text{i}}} \n Label:{{label{i}}}\n"
    prompt_template += "Sentence: {Context} \n Label:<|end|><|assistant|>"
    return prompt_template

# Function to generate prompts based on the selected examples
def generate_prompt_with_samples(df, text_column, label_column, instruction_column,num_samples):
    # Generate dynamic prompt template based on the number of samples
    INSTRUCTION_PROMPT_TEMPLATE = generate_dynamic_prompt_template(df.loc[0,'instruction'],num_samples)

    RESPONSE_TEMPLATE = """\
    {answer}"""
    # Ensure there are enough examples
    if len(df) < num_samples:
        raise ValueError(f"The DataFrame must contain at least {num_samples} examples for each label.")

    # Create a dictionary to store the values to be formatted into the template
    format_dict = {}
    format_dict['instruction'] = df.loc[0, instruction_column]  # Use instruction from the first row
    for i in range(num_samples):
        format_dict[f'text{i + 1}'] = df.loc[i, text_column]
        format_dict[f'label{i + 1}'] = df.loc[i, label_column]
    format_dict['Context'] = "{Context}"  #

    # Generate the prompt with the samples
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(**format_dict)

    return prompt

# Specify the number of samples to retrieve for each label
num_samples = 1  # Change this value as needed

# Select random examples based on the 'label' column and the specified number of samples
random_examples = select_random_examples(icl_df, 'label', num_samples)

# Generate prompt with the selected examples
prompt_with_samples = generate_prompt_with_samples(random_examples, 'text', 'label', 'instruction',len(random_examples))

# Display the generated prompt with placeholders for text samples
print("Generated Prompt with Samples:")
print(prompt_with_samples)

Generated Prompt with Samples:
    <|user|>
    you are a skilled financial analyst specialized in detecting market sentiment from news sources.Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.
 Return only a single word, either Positive or Negative or Neutral. See below a couple of examples
Sentence:The company said production volumes so far indicate the circuit is capable of the targeted output rate of 60,000 tonnes per day , or 22 million tonnes a year . 
 Label:neutral
Sentence:Entering long Lockheed Martin $LMT at Thursday's open. http://stks.co/eXtA 
 Label:positive
Sentence:However , the growth margin slowed down due to the financial crisis . 
 Label:negative
Sentence: {Context} 
 Label:<|end|><|assistant|>


In [ ]:
# config = json.load(open('./config.json', 'r'))
# config
TASK = "cls_sentiment" # ["re_finred", "cls_headline", "sentiment_fbp_fiqasa","cls_fomc"]
VERSION = "v1"
#MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
#MODEL_ID = "microsoft/Phi-3-mini-4k-instruct" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
DATASET_ID = "Sorour/sentiment-train" # "Sorour/sentiment-train", "Sorour/headline","Sorour/train-cls"
PADDING_SIDE = "right"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
def eval(y_pred, y_true):# Calculate evaluation params
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='weighted')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='weighted')

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Print the results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return (accuracy, precision, recall, f1)

In [ ]:
def get_inference_result(prompt_with_samples,df, model_pipe, output_name, params):
# Create an empty list to store the outputs
    outputs_list = []

    for idx, row  in df.iterrows():
        print(idx)
        print(row['text'])
        print(prompt_with_samples.format(Context=row['text']))
        result = model_pipe(prompt_with_samples.format(Context=row['text']), do_sample=params['do_sample'],
                            max_new_tokens=params['max_new_tokens'], temperature=params['temperature'], top_k=params['top_k'])
        outputs=result[0]["generated_text"]
        print(outputs)
        outputs_list.append(outputs)
    # Add the outputs_list as a new column to the DataFrame
    df[output_name] = outputs_list
    return df

In [ ]:
DATASET_ID="Sorour/sentiment-test-fpb"
df = load_dataset(DATASET_ID)

Generating test split:   0%|          | 0/970 [00:00<?, ? examples/s]

In [ ]:
df_test = pd.DataFrame(df['test'])
df_test

,label,text,instruction
0,neutral,The purchase sum is about EUR 10mn US$ 12.97 mn .,you are a skilled financial analyst specialize...
1,positive,The Dutch broker noted that Nokian Tyres repor...,you are a skilled financial analyst specialize...
2,negative,The first ship has been delayed and is estimat...,you are a skilled financial analyst specialize...
3,neutral,`` This transaction supports Huhtamaki 's stra...,you are a skilled financial analyst specialize...
4,positive,Finnish KCI Konecranes has been awarded an ord...,you are a skilled financial analyst specialize...
...,...,...,...
965,positive,Finnish operator Elisa and Aker Yards have sig...,you are a skilled financial analyst specialize...
966,positive,"Profit for the period was EUR 9.8 mn , up from...",you are a skilled financial analyst specialize...
967,neutral,Neste Oil said that while results from its bio...,you are a skilled financial analyst specialize...
968,neutral,The first group of customers to be trained wil...,you are a skilled financial analyst specialize...


In [ ]:
params = {
    "do_sample":False,
    "max_new_tokens":500,
    "temperature":0.1,
    "top_k":50
}
colname = 'predict_llama3'

# df_test_fiqasa = get_inference_result(df_test_fiqasa, ft_pipe, output_name = colname, params=params)
base_model_pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=False)
df_test= get_inference_result(prompt_with_samples,df_test, base_model_pipe, output_name = colname, params=params)

0
The purchase sum is about EUR 10mn US$ 12.97 mn .
    <s>[INST] <<SYS>>
    You are a skilled financial analyst specialized in detecting market sentiment from news sources.Your task is to evaluate the sentiment of the following sentence and assign it one of the labels: Positive, Negative, or Neutral.
 Return only a single word, either Positive or Negative or Neutral<</SYS>>. See below a couple of examples:
Sentence:Its annual capacity is some 10,000 MW . 
 Label:neutral
Sentence:Finnair expands : Finnair plans to add more flights on routes to Hong Kong , India and Russia . 
 Label:positive
Sentence:And U.S. energy executives say high steel prices are threatening energy exploration . 
 Label:negative
Sentence:The purchase sum is about EUR 10mn US$ 12.97 mn . 
 Label:[/INS]
 neutral

Sentence:The company reported a net loss of $10 million for the quarter.
 Label:negative

Sentence:The company's stock price has increased by 15% in the past month.
 Label:positive

Sentence:The new policy

KeyboardInterrupt: 

In [ ]:
file_path='/content/drive/MyDrive/results-v2/fomc_phi3_ft_alldata.csv'

df_test.to_csv(file_path, index=False)

In [ ]:
y_pred = df_test['predict_phi3_ft']
y_true = df_test['label']
(accuracy, precision, recall, f1) = eval(y_pred, y_true)